# Desafio 4

## 1. Introdução ao Pandas 

Já visto nas aulas anteriores.

## 2. Carregar o arquivo JSON

Ver slides 57 e 58.


In [ ]:
import simplejson as json
import pandas as pd

In [ ]:
dados = []
with open('tweets_5min.json') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha))

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [ ]:
print(len(df))

## 3. Definir o que iremos salvar!

Ver slides 61 até 64.

Vamos salvar:

* text
* created_at
* coordinates
* retweet_count
* user -> screen_name
* user -> location
* user -> lang
* user -> followers_count



In [ ]:
print("--> Dados do tweet")
print(df['text'][0])
print(df['created_at'][0])
print(df['coordinates'][0])
print(df['retweet_count'][0])

print("\n--> Dados do usuário")
print(df['user'][0]['screen_name'])
print(df['user'][0]['location'])
print(df['user'][0]['lang'])
print(df['user'][0]['followers_count'])


Agora vamos criar uma lista chamada colunas para salvar os nomes das colunas que queremos ter em nosso DataFrame.

In [ ]:
colunas = ['text', 'created_at', 'coordinates', 'retweet_count', 'screen_name', 
           'location', 'lang', 'followers_count']

In [ ]:
print(len(colunas))

Vamos criar um dataframe auxiliar para salvar somente os dados que queremos.

In [ ]:
df_aux = pd.DataFrame(columns=colunas)
df_aux

Agora iremos adicionar apenas uma linha para entender como essa estrutura funciona.

Lembre-se que estamos recuperando informações do ```user```, sendo que os valores estão salvos em uma única coluna.

In [ ]:
dados = [df['text'][0],
         df['created_at'][0],
         df['coordinates'][0],
         df['retweet_count'][0],
         df['user'][0]['screen_name'],
         df['user'][0]['location'],
         df['user'][0]['lang'],
         df['user'][0]['followers_count']
        ]

In [ ]:
series_aux = pd.Series(dados, index=colunas)
df_aux = df_aux.append(series_aux, ignore_index=True)

É importante notar que se a cedula acima for executada mais de uma vez, o append irá adicionar repetidos. Tome cuidado!

In [ ]:
df_aux

## 4. Entender o módulo Geopy e criar função que retorna a latitude e longitude

Ver slides 67 até 70.


In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
print(dir(Nominatim))

In [ ]:
geolocalizador = Nominatim()
localizacao = geolocalizador.geocode("Rua do Rocio, 109, SP")
print(localizacao)

In [ ]:
print(localizacao.address)

In [ ]:
print(localizacao.latitude, localizacao.longitude)

In [ ]:
print(localizacao.raw)

**Exercício - **```Crie uma função que receba uma localização e retorne a latitude e longitude. Lembre-se de tratar possíves erros de conexão.```

In [ ]:
from geopy.geocoders import Nominatim
def pegar_lat_long(local):
    # seu código aqui
    

In [ ]:
pegar_lat_long('Sao Paulo, SP')

## 5. Preparar os dados para a visualização

Ver slides 73 até 74.

### Hashtags

Agora iremos criar uma função que irá salvar as hashtags utilizadas no texto.

In [ ]:
palavras = df['text'][0]
palavras.split()

In [ ]:
for palavra in palavras.split():
    if palavra.startswith('#'):
        print(palavra)

** Exercicio** - ```Crie uma função que salvar as hashtags separadas por um espaço em branco. Exemplo:```

**Texto de entrada:** ```Eu estou #programando em #python```

**Saida:** ```'#programando #python'```
    

In [ ]:
def salvar_hashtags(texto):
    #seu código aqui

In [ ]:
salvar_hashtags(df['text'][0])

### Criar as novas colunas no DataFrame

Vamos criar as 3 novas colunas em nosso DataFrame.



In [ ]:
colunas = ['text', 'created_at', 'coordinates', 'retweet_count', 
           'screen_name', 'location', 'lang', 'followers_count', 
           'lat', 'long', 'hastags']

In [ ]:
df_aux = pd.DataFrame(columns=colunas)
len(df_aux.columns)

In [ ]:
df_aux

Novamente vamos adicionar apenas um único item para verificar se está tudo correto.

In [ ]:
latlong = pegar_lat_long(df['user'][0]['location'])
latlong

In [ ]:
dados = [
    df['text'][0],
    df['created_at'][0],
    df['coordinates'][0],
    df['retweet_count'][0],
    df['user'][0]['screen_name'],
    df['user'][0]['location'],
    df['user'][0]['lang'],
    df['user'][0]['followers_count'],
    0, #latlong[0],
    0, #latlong[1],
    salvar_hashtags(df['text'][0])
    ]

In [ ]:
dados

In [ ]:
series_aux = pd.Series(dados, index=colunas)
df_aux = df_aux.append(series_aux, ignore_index=True)

In [ ]:
df_aux

### Repetir esse processo para todos os tweets que foram salvos!

Até agora só trabalhamos com uma única entrada de dados, que foi o primeiro twitter salvo no arquivo.

Agora precisamos verificar quais informações são realmente importante e não podem ser vazias.

Primeiro precisamos percorrer todo o conjunto de dados, linha a linha. Como podemos fazer isso?

In [ ]:
df_aux = pd.DataFrame(columns=colunas)
df_aux

In [ ]:
print(len(df))

In [ ]:
import time

In [ ]:
t0 = time.time()
for i in range(0, len(df)):
    if df['user'][i]['location'] != None:
        tem_lat_long = pegar_lat_long(df['user'][i]['location'])
        if tem_lat_long != 0:
            dados = [
                df['text'][i],
                df['created_at'][i],
                df['coordinates'][i],
                df['retweet_count'][i],
                df['user'][i]['screen_name'],
                df['user'][i]['location'],
                df['user'][i]['lang'],
                df['user'][i]['followers_count'],
                tem_lat_long[0],
                tem_lat_long[1],
                salvar_hashtags(df['text'][i])
            ]
            print(i,end=" ")
            series = pd.Series(dados,index=colunas)
            df_aux = df_aux.append(series, ignore_index=True)
tf = time.time() - t0
print("\n\nTempo total para o parse foi de {} minutos".format(round(tf/60, 3)))

In [ ]:
len(df_aux)

In [ ]:
df_aux.head()

## 6. Salvar os dados em CSV

In [ ]:
print(df_aux.count())

In [ ]:
df_aux.to_csv('tweets_5min.csv', sep=';', encoding='utf-8', index=False)